In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)


In [3]:
df_procesada = pd.read_csv("../data/interim/datos_limpios.csv", encoding="latin-1", sep=";")
df_procesada["FECHA 1º evaluación"] = pd.to_datetime(
    df_procesada["FECHA 1º evaluación"], format="%Y-%m-%d"
)


# Analisis de Egresos por Anio

In [4]:
egresos_por_anio = (
    df_procesada.groupby(["ANIO_PRIMERA_EVALUACION", "MES_PRIMERA_EVALUACION"])[
        "DIAGNOSTICO PRINCIPAL", "cie"
    ]
    .value_counts()
    .reset_index(name="conteo")
)

egresos_por_anio = build_features.agregar_info_codigo_cie(egresos_por_anio, "cie")

In [5]:
egresos_por_anio.to_excel("../data/processed/conteo_diagnostico.xlsx", index=False)

# Analisis de Sexo por Anio

In [6]:
df_procesada["SEXO"].value_counts()

f    1834
m    1718
Name: SEXO, dtype: int64

In [11]:
(df_procesada.groupby(["ANIO_PRIMERA_EVALUACION", "SEXO"])[
        "DIAGNOSTICO PRINCIPAL", "cie"
    ]
    .value_counts()
    .reset_index(name="conteo"))

,ANIO_PRIMERA_EVALUACION,SEXO,DIAGNOSTICO PRINCIPAL,cie,conteo
0,1900.0,f,civ,Q21.0,1
1,1900.0,f,cia,Q21.1,1
2,1905.0,f,canal av,Q21.2,1
3,1990.0,f,canal av,Q21.2,1
4,1999.0,f,atresia pulmonar,Q22.0,1
...,...,...,...,...,...
921,2023.0,m,valvula aortica bicuspide,Q23.8,1
922,2023.0,m,canal av,Q21.2,1
923,2023.0,m,cia,Q21.1,1
924,2023.0,m,civ,Q21.0,1


In [7]:
atendidos_antes_de_1900 = df_procesada[
    df_procesada["FECHA 1º evaluación"] < pd.to_datetime("2000-01-01", yearfirst=True)
]

In [8]:
atendidos_antes_de_1900.to_csv("../data/interim/atendidos_antes_del_2000.xlsx", encoding="latin-1",
                               sep=";")